# Data Cleaning Notebook

In [893]:
# data cleaning and creation imports
import pandas as pd
import os
import glob
import itertools

# mitigate warnings
import warnings
warnings.filterwarnings("ignore")

# internal imports
import sys, os
sys.path.append(os.path.abspath(".."))
from src.utils import nba_teams, team_map

### Utils Direcrtory Checks

In [894]:
print(len(nba_teams))   # should be 30
print(team_map["LAL"])  # "Los Angeles Lakers"

30
Los Angeles Lakers


## Creating Compressed Data

In [895]:
# # THE CODE TO CREATE POROCESSED GZIP FILES
# root_dir = "/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats"

# for subdir, _, files in os.walk(root_dir):
#     for file in files:
#         if file.endswith(".csv"):
#             file_path = os.path.join(subdir, file)
#             gz_path = file_path + ".gz"

#             try:
#                 df = pd.read_csv(file_path)

#                 if df.empty:
#                     print(f"Skipping empty file: {file_path}")
#                     continue

#                 print(f"Compressing {file_path} -> {gz_path}")
#                 df.to_csv(gz_path, index=False, compression="gzip")

#             except pd.errors.EmptyDataError:
#                 print(f"Skipping empty file (no data): {file_path}")


## CSV Cleaning

For each csv file, I have already gone in and manually lightly cleaned the data with Microsoft Excel; however, for deeper cleaning I will use python libraries here in this ipynb file.

In Microsoft Excel, I:
- eliminated unnecessary rows and columns from the dataset
- added in the columns necessary for aggregation and merging
- formatted everything correctly to prepare it for python data manipulation

In [896]:
team_records_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-records.csv")
team_records_df.head(5)

,Rk,Team,Home,Road,E,W,Pre-ASG,Post-ASG,Season
0,1,Oklahoma City Thunder,36-6,32-8,29-1,39-13,44-10,24-4,2025
1,2,Cleveland Cavaliers,34-7,30-11,41-11,23-7,44-10,20-8,2025
2,3,Boston Celtics,28-13,33-8,39-13,22-8,39-16,22-5,2025
3,4,Houston Rockets,29-12,23-18,21-9,31-21,34-21,18-9,2025
4,5,New York Knicks,27-14,24-17,34-18,17-13,36-18,15-13,2025


For this team_records_df, I need to make it so that the wins and loss columns are seperated.

In [897]:
# Columns to split
split_cols = ["Home", "Road", "E", "W", "Pre-ASG", "Post-ASG"]

# Loop through and split each column into wins/losses
for col in split_cols:
    team_records_df[[f"{col}_W", f"{col}_L"]] = team_records_df[col].str.split("-", expand=True).astype(int)

# remove original columns
team_records_df = team_records_df.drop(columns=split_cols)

# Check results
team_records_df.head()


,Rk,Team,Season,Home_W,Home_L,Road_W,Road_L,E_W,E_L,W_W,W_L,Pre-ASG_W,Pre-ASG_L,Post-ASG_W,Post-ASG_L
0,1,Oklahoma City Thunder,2025,36,6,32,8,29,1,39,13,44,10,24,4
1,2,Cleveland Cavaliers,2025,34,7,30,11,41,11,23,7,44,10,20,8
2,3,Boston Celtics,2025,28,13,33,8,39,13,22,8,39,16,22,5
3,4,Houston Rockets,2025,29,12,23,18,21,9,31,21,34,21,18,9
4,5,New York Knicks,2025,27,14,24,17,34,18,17,13,36,18,15,13


In [898]:
team_stats_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-stats.csv")
team_stats_df.head().shape

(5, 28)

### Optimal Dataframe Construction

I now want to merge the dataframes that contain the team records and team stats (FG%, FGA, MIN) position. I will do this by merging by team and season.

In [899]:
# Merge on Team and Season
team_df = pd.merge(
    team_stats_df,
    team_records_df,
    on=["Team", "Season"],
    how="outer"  
)

# Check result
print(team_df.head())
print(f"-----")
print(f"The shape of the dataframe:", team_df.shape)

   Season           Team  GP   W   L   WIN%   Min    PTS   FGM   FGA  ...  \
0    2016  Atlanta Hawks  82  48  34  0.585  48.4  102.8  38.6  84.4  ...   
1    2017  Atlanta Hawks  82  43  39  0.524  48.5  103.2  38.1  84.4  ...   
2    2018  Atlanta Hawks  82  24  58  0.293  48.1  103.4  38.2  85.5  ...   
3    2019  Atlanta Hawks  82  29  53  0.354  48.4  113.3  41.4  91.8  ...   
4    2020  Atlanta Hawks  67  20  47  0.299  48.6  111.8  40.6  90.6  ...   

   Road_W  Road_L  E_W  E_L  W_W  W_L  Pre-ASG_W  Pre-ASG_L  Post-ASG_W  \
0      21      20   29   23   19   11         31         24          17   
1      20      21   30   22   13   17         32         24          15   
2       8      33   12   40   18   12         18         41          17   
3      29      12   16   36   13   17         19         39          14   
4      27       6   11   32   15    9         15         41           6   

   Post-ASG_L  
0          10  
1          11  
2           6  
3          10  
4     

In [900]:
draft_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/draft.csv")
draft_df.head()

,Pk,Tm,Season
0,1,DAL,2025
1,2,SAS,2025
2,3,PHI,2025
3,4,CHO,2025
4,5,UTA,2025


In [901]:
coach_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/coach.csv")
coach_df.head(5)

,Coach,Tm,Yw/Franch,YOverall,CareerW,CareerL,CareerW%,Season
0,Quin Snyder,ATL,3,11,458,363,0.558,2025
1,Joe Mazzulla,BOS,3,3,182,64,0.740,2025
2,Jordi Fernandez,BRK,1,1,26,56,0.317,2025
3,Billy Donovan,CHI,5,10,438,362,0.548,2025
4,Charles Lee,CHO,1,1,19,63,0.232,2025


In [902]:
# Replace acronyms with full names
coach_df["Team"] = coach_df["Tm"].map(team_map)
draft_df["Team"] = draft_df["Tm"].map(team_map)

# Drop the old acronym column
coach_df = coach_df.drop(columns=["Tm"])
draft_df = draft_df.drop(columns=["Tm"])

# Check result
print(coach_df.head())
print(f"The shape of the coaches df:", coach_df.shape)
print(f"----------------------------------------------")
print(draft_df.head())
print(f"The shape of the draft df:", draft_df.shape)


             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1     Joe Mazzulla          3         3      182       64     0.740    2025   
2  Jordi Fernandez          1         1       26       56     0.317    2025   
3    Billy Donovan          5        10      438      362     0.548    2025   
4      Charles Lee          1         1       19       63     0.232    2025   

                Team  
0      Atlanta Hawks  
1     Boston Celtics  
2      Brooklyn Nets  
3      Chicago Bulls  
4  Charlotte Hornets  
The shape of the coaches df: (328, 8)
----------------------------------------------
   Pk  Season                Team
0   1    2025    Dallas Mavericks
1   2    2025   San Antonio Spurs
2   3    2025  Philadelphia 76ers
3   4    2025   Charlotte Hornets
4   5    2025           Utah Jazz
The shape of the draft df: (659, 3)


In order to alleviate duplicate rows, I will turn the draft pick column into two columns, those being the sum of First Round (Picks 1-30) and Second Round (Picks 31-60).

In [903]:
# Ensure Pk is numeric
draft_df["Pk"] = pd.to_numeric(draft_df["Pk"], errors="coerce")

# Create round indicator columns
draft_df["FirstRound"] = (draft_df["Pk"] <= 30).astype(int)
draft_df["SecondRound"] = (draft_df["Pk"] > 30).astype(int)

# Aggregate to collapse duplicate rows (team-season level)
draft_df_cleaned = (
    draft_df.groupby(["Season", "Team"], as_index=False)
    .agg(
        FirstRoundPicks=("FirstRound", "sum"),
        SecondRoundPicks=("SecondRound", "sum")
    )
)

# Build full season-team grid
all_seasons = draft_df_cleaned["Season"].unique()
full_index = pd.DataFrame(
    list(itertools.product(all_seasons, nba_teams)),
    columns=["Season", "Team"]
)

In [904]:
# Merge to ensure all 30 teams per season
draft_df_full = pd.merge(full_index, draft_df_cleaned, on=["Season", "Team"], how="left")

# Fill missing picks with 0
draft_df_full["FirstRoundPicks"] = draft_df_full["FirstRoundPicks"].fillna(0).astype(int)
draft_df_full["SecondRoundPicks"] = draft_df_full["SecondRoundPicks"].fillna(0).astype(int)

# Drop original Pk column if still present
if "Pk" in draft_df_full.columns:
    draft_df_full = draft_df_full.drop(columns=["Pk"])

# Sanity check
print("Original shape:", draft_df.shape)
print("Fixed shape:", draft_df_full.shape)
print("2025 missing teams:", set(nba_teams) - set(draft_df_full[draft_df_full["Season"] == 2025]["Team"].unique()))
print(draft_df_full.head())

Original shape: (659, 5)
Fixed shape: (330, 4)
2025 missing teams: set()
   Season               Team  FirstRoundPicks  SecondRoundPicks
0    2015      Atlanta Hawks                1                 2
1    2015     Boston Celtics                2                 2
2    2015      Brooklyn Nets                1                 1
3    2015  Charlotte Hornets                1                 1
4    2015      Chicago Bulls                1                 0


In [905]:
# For draft_df leaks
draft_2024 = draft_df_full[draft_df_full["Season"] == 2024]["Team"].unique()
draft_2025 = draft_df_full[draft_df_full["Season"] == 2025]["Team"].unique()

print("draft_df → Missing in 2024:", set(nba_teams) - set(draft_2024))
print("draft_df → Missing in 2025:", set(nba_teams) - set(draft_2025))


# For coach_df leaks
coach_2024 = coach_df[coach_df["Season"] == 2024]["Team"].unique()
coach_2025 = coach_df[coach_df["Season"] == 2025]["Team"].unique()

print("coach_df → Missing in 2024:", set(nba_teams) - set(coach_2024))
print("coach_df → Missing in 2025:", set(nba_teams) - set(coach_2025))

draft_df → Missing in 2024: set()
draft_df → Missing in 2025: set()
coach_df → Missing in 2024: set()
coach_df → Missing in 2025: set()


I now want to merge the dataframes that contain the length of the current coach and their draft position. I will do this by merging by team and season.

In [906]:
# Merge on Team and Season
front_office_df = pd.merge(
    coach_df,
    draft_df_full,
    on=["Team", "Season"],
    how="inner"  
)

print(front_office_df.head())
print(f"-----")
print(f"The shape of the dataframe:", front_office_df.shape)

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1     Joe Mazzulla          3         3      182       64     0.740    2025   
2  Jordi Fernandez          1         1       26       56     0.317    2025   
3    Billy Donovan          5        10      438      362     0.548    2025   
4      Charles Lee          1         1       19       63     0.232    2025   

                Team  FirstRoundPicks  SecondRoundPicks  
0      Atlanta Hawks                2                 0  
1     Boston Celtics                1                 1  
2      Brooklyn Nets                4                 1  
3      Chicago Bulls                1                 1  
4  Charlotte Hornets                1                 2  
-----
The shape of the dataframe: (328, 10)


I now need to add a column to this front offfice dataframe that counts the amount of coaches each team has in each season.

In [907]:
# Count how many distinct coaches each Team had in each Season
coach_counts = coach_df.groupby(["Team", "Season"])["Coach"].nunique().reset_index()
coach_counts.rename(columns={"Coach": "Coach_Count"}, inplace=True)

# Merge that count into your front_office_df
front_office_df = pd.merge(
    front_office_df,
    coach_counts,
    on=["Team", "Season"],
    how="left"
)

print(front_office_df.head())
print(f"-----")
print(f"The shape of the dataframe:", front_office_df.shape)

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1     Joe Mazzulla          3         3      182       64     0.740    2025   
2  Jordi Fernandez          1         1       26       56     0.317    2025   
3    Billy Donovan          5        10      438      362     0.548    2025   
4      Charles Lee          1         1       19       63     0.232    2025   

                Team  FirstRoundPicks  SecondRoundPicks  Coach_Count  
0      Atlanta Hawks                2                 0            1  
1     Boston Celtics                1                 1            1  
2      Brooklyn Nets                4                 1            1  
3      Chicago Bulls                1                 1            1  
4  Charlotte Hornets                1                 2            1  
-----
The shape of the dataframe: (328, 11)


In [908]:
# For front office data leaks
fo_2024 = front_office_df[front_office_df["Season"] == 2024]["Team"].unique()
fo_2025 = front_office_df[front_office_df["Season"] == 2025]["Team"].unique()

print("Missing in 2024:", set(nba_teams) - set(fo_2024))
print("Missing in 2025:", set(nba_teams) - set(fo_2025))

Missing in 2024: set()
Missing in 2025: set()


In [909]:
front_office_df[front_office_df["Coach_Count"] > 1]

,Coach,Yw/Franch,YOverall,CareerW,CareerL,CareerW%,Season,Team,FirstRoundPicks,SecondRoundPicks,Coach_Count
7,Michael Malone,10,12,510,394,0.564,2025,Denver Nuggets,0,0,2
8,David Adelman,1,1,3,0,1.000,2025,Denver Nuggets,0,0,2
15,Taylor Jenkins,6,6,250,214,0.539,2025,Memphis Grizzlies,1,2,2
16,Tuomas Iisalo,1,1,4,5,0.444,2025,Memphis Grizzlies,1,2,2
27,Mike Brown,3,11,454,304,0.599,2025,Sacramento Kings,0,1,2
28,Doug Christie,1,1,27,24,0.529,2025,Sacramento Kings,0,1,2
29,Gregg Popovich,29,29,1390,824,0.628,2025,San Antonio Spurs,2,1,2
30,Mitch Johnson,1,1,32,45,0.416,2025,San Antonio Spurs,2,1,2
36,Jacque Vaughn,3,6,129,226,0.363,2024,Brooklyn Nets,0,0,2
37,Kevin Ollie,1,1,11,17,0.393,2024,Brooklyn Nets,0,0,2


I will now add the teams payroll data into the front office dataframe. This will just give my front office dataset that last bit of kick to make accurate predictions on the future of NBA Teams!

In [910]:
team_payroll_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-payroll.csv")
print(team_payroll_df.head())

                Team      2016       2017       2018       2019       2020  \
0      Atlanta Hawks  71661760   96315163   99992696   79180081  110702618   
1     Boston Celtics  77141919   93465326  115284776  125334993  117759332   
2      Brooklyn Nets  80258302   82391482   95475397  118850600  118889943   
3  Charlotte Hornets  76860006  103054004  117382664  121427859   96552033   
4      Chicago Bulls  87073838   92522306   90466801  112598201  112601901   

        2021       2022       2023       2024       2025       2026  \
0  118804016  135166020  149836313  159153393  170057021  190318494   
1  132931565  136557646  178633307  186940921  195348491  204311991   
2  170444633  174811922  159566723  155015136  168312896  139805782   
3  108218809  122139566  125874047  140774484  168575524  180776048   
4  128963580  136083814  151964990  165630436  165722496  176052328   

        2027       2028         2029  
0  120120521   62041800   45919890.0  
1  170903314  176353230  1

It seems I have mispelled some of the teams names and need to find out which ones. This is one of the errors expected from manual insertion of data.

In [911]:
team_payroll_df.info

<bound method DataFrame.info of                       Team       2016       2017       2018       2019  \
0            Atlanta Hawks   71661760   96315163   99992696   79180081   
1           Boston Celtics   77141919   93465326  115284776  125334993   
2            Brooklyn Nets   80258302   82391482   95475397  118850600   
3        Charlotte Hornets   76860006  103054004  117382664  121427859   
4            Chicago Bulls   87073838   92522306   90466801  112598201   
5      Cleveland Cavaliers  108300458  128522489  137722926  123255073   
6         Dallas Mavericks   75397067  102354966   85440245   86958881   
7           Denver Nuggets   72589023   83836460  107543599  118327016   
8          Detroit Pistons   77256014  106492988  119773191  126557932   
9    Golden State Warriors   93669566  101584835  137610134  146291276   
10         Houston Rockets   87504058   90956067  119905532  126474100   
11          Indiana Pacers   71605233   90279072   95271736  110724804   
12    

In [912]:
# Reshape payroll_df from wide to long
payroll_long = team_payroll_df.melt(
    id_vars=["Team"],
    var_name="Season",
    value_name="Payroll"
)

# Convert Season column to int
payroll_long["Season"] = payroll_long["Season"].astype(int)

# Merge with front_office_df on Team and Season
front_office_df = pd.merge(
    front_office_df,
    payroll_long,
    on=["Team", "Season"],
    how="left"
)

print(front_office_df.head())
print(f"Shape: {front_office_df.shape}")

             Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  Season  \
0      Quin Snyder          3        11      458      363     0.558    2025   
1     Joe Mazzulla          3         3      182       64     0.740    2025   
2  Jordi Fernandez          1         1       26       56     0.317    2025   
3    Billy Donovan          5        10      438      362     0.548    2025   
4      Charles Lee          1         1       19       63     0.232    2025   

                Team  FirstRoundPicks  SecondRoundPicks  Coach_Count  \
0      Atlanta Hawks                2                 0            1   
1     Boston Celtics                1                 1            1   
2      Brooklyn Nets                4                 1            1   
3      Chicago Bulls                1                 1            1   
4  Charlotte Hornets                1                 2            1   

       Payroll  
0  170057021.0  
1  195348491.0  
2  168312896.0  
3  165722496.0  
4  1685

In [913]:
front_office_df[front_office_df["Payroll"].isna()]

,Coach,Yw/Franch,YOverall,CareerW,CareerL,CareerW%,Season,Team,FirstRoundPicks,SecondRoundPicks,Coach_Count,Payroll


### Strength of Schedule Calculation

Next, I need to use the dataframe containing the teams records against themselves to calculate each teams strength of schedule, giving me an even more accurate prediction of the amount of wins they will have in the 2025-26 season.

The formula I will use for the calculation is:
∑(Opposing Team Win Pct * Games vs the Opponent) / Total Games


In [914]:
team_sos_df = pd.read_csv("/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/team-stats/team-sos.csv")
team_sos_df = team_sos_df.rename(columns=team_map)

print(team_sos_df.head())

   Rk               Team Atlanta Hawks Boston Celtics Brooklyn Nets  \
0   1      Atlanta Hawks           NaN            2-1           2-1   
1   2     Boston Celtics           1-2            NaN           4-0   
2   3      Brooklyn Nets           1-2            0-4           NaN   
3   4      Chicago Bulls           2-2            1-3           2-1   
4   5  Charlotte Hornets           0-4            0-4           1-3   

  Chicago Bulls Charlotte Hornets Cleveland Cavaliers Dallas Mavericks  \
0           2-2               4-0                 2-1              0-2   
1           3-1               4-0                 2-2              1-1   
2           1-2               3-1                 0-4              1-1   
3           NaN               3-1                 0-4              0-2   
4           1-3               NaN                 0-4              1-1   

  Denver Nuggets  ... Orlando Magic Philadelphia 76ers Phoenix Suns  \
0            0-2  ...           2-2                3-0   

In [915]:
# Build mapping (Team+Season → WinPct) from team_df
opp_winpct = dict(zip(zip(team_df["Team"], team_df["Season"]), team_df["WIN%"]))

# Calculate Strength of Schedule
sos_list = []

for _, row in team_sos_df.iterrows():
    team = row["Team"]
    season = row["Season"]

    total_weighted = 0
    total_games = 0

    for opp, record in row.items():
        if opp in ["Rk", "Team", "Season"] or pd.isna(record):
            continue

        try:
            wins, losses = map(int, str(record).split("-"))
        except:
            continue

        games = wins + losses
        opp_team = opp
        key = (opp_team, season)

        if key in opp_winpct:
            total_weighted += opp_winpct[key] * games
            total_games += games

    sos = total_weighted / total_games if total_games > 0 else None
    sos_list.append([team, season, sos])

# --- Step 3: Build SOS DataFrame ---
sos_df = pd.DataFrame(sos_list, columns=["Team", "Season", "SOS"])

print(sos_df.head())
print(f"Shape: {sos_df.shape}")

                Team  Season       SOS
0      Atlanta Hawks    2025  0.495012
1     Boston Celtics    2025  0.479110
2      Brooklyn Nets    2025  0.507207
3      Chicago Bulls    2025  0.495756
4  Charlotte Hornets    2025  0.504085
Shape: (300, 3)


### Player Stats Calculations

Now that I have calculated the strength of schedule, I need to calculate a few more player stats that I can add to my final dataframe. The stats I will be using from these player stats are:
- average age
- average points of top 10 players
- production score: (Points + PlUS/MINUS) / Minutes Played
- injury rate: (82 × Roster Size) − Total Games Played

First, I need to parse through my player-stats folder and gather all of the player statistics I scraped from Basketball Reference.

In [916]:
teams_2024 = front_office_df[front_office_df["Season"] == 2024]["Team"].unique()
teams_2025 = front_office_df[front_office_df["Season"] == 2025]["Team"].unique()

print("Missing in 2024:", set(nba_teams) - set(teams_2024))
print("Missing in 2025:", set(nba_teams) - set(teams_2025))

Missing in 2024: set()
Missing in 2025: set()


In [917]:
# Path to your player-stats folder
path = "/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/player-stats/"

all_files = glob.glob(os.path.join(path, "*-player-stats.csv"))

dfs = []

for file in all_files:
    season = int(os.path.basename(file).split("-")[0])
    
    df = pd.read_csv(file)
    
    df["Season"] = season
    
    dfs.append(df)

players_df = pd.concat(dfs, ignore_index=True)

print(players_df.head())
print("Shape:", players_df.shape)
print("Seasons in dataset:", players_df["Season"].unique())
print(players_df.info())


    Rk                 Player   Age Team Pos     G    GS    MP    FG   FGA  \
0  1.0            Joel Embiid  27.0  PHI   C  68.0  68.0  33.8   9.8  19.6   
1  2.0           LeBron James  37.0  LAL   C  56.0  56.0  37.2  11.4  21.8   
2  3.0  Giannis Antetokounmpo  27.0  MIL  PF  67.0  67.0  32.9  10.3  18.6   
3  4.0           Kevin Durant  33.0  BRK  PF  55.0  55.0  37.2  10.5  20.3   
4  5.0            Luka Dončić  22.0  DAL  PG  65.0  65.0  35.4   9.9  21.6   

   ...   TRB  AST  STL  BLK  TOV   PF   PTS                 Awards  \
0  ...  11.7  4.2  1.1  1.5  3.1  2.7  30.6            MVP-2ASNBA2   
1  ...   8.2  6.2  1.3  1.1  3.5  2.2  30.3           MVP-10ASNBA3   
2  ...  11.6  5.8  1.1  1.4  3.3  3.2  29.9  MVP-3DPOY-6ASNBA1DEF1   
3  ...   7.4  6.4  0.9  0.9  3.5  2.1  29.9           MVP-10ASNBA2   
4  ...   9.1  8.7  1.2  0.6  4.5  2.2  28.4            MVP-5ASNBA1   

   Player-additional  Season  
0          embiijo01    2022  
1          jamesle01    2022  
2          antetg

Next, I need to make a new dataframe with the calculations I had explained earlier. Then I will take these calculations and add them to the team dataframe.

In [918]:
# --- Step 1: drop rows for multi-team players (2TM, 3TM, etc.) ---
players_df = players_df[~players_df["Team"].isin(["2TM", "3TM", "4TM"])]

# --- Step 2: standardize team names ---
players_df["Team"] = players_df["Team"].map(team_map).fillna(players_df["Team"])

# --- Step 3: create production score per player ---
# Avoid divide-by-zero issues
players_df["Production_Score"] = (players_df["PTS"] + players_df.get("PLUS_MINUS", 0)) / players_df["MP"].replace(0, 1)

# --- Step 4: aggregate to team-season level ---
player_stats_df = (
    players_df.groupby(["Season", "Team"])
    .apply(lambda x: pd.Series({
        "avg_age": x["Age"].mean(),
        "avg_pts_top10": x.sort_values("PTS", ascending=False).head(10)["PTS"].mean(),
        "avg_production_score": x["Production_Score"].mean(),
        "injury_rate": (82 * len(x) - x["G"].sum()) / (82 * len(x))  # est. based on games played
    }))
    .reset_index()
)

# --- Step 5: sanity check ---
print(player_stats_df.head())
print("Shape:", player_stats_df.shape)


   Season               Team    avg_age  avg_pts_top10  avg_production_score  \
0    2016      Atlanta Hawks  27.470588          10.52              0.377504   
1    2016     Boston Celtics  24.500000          11.20              0.428270   
2    2016      Brooklyn Nets  26.000000          11.39              0.389805   
3    2016  Charlotte Hornets  25.941176          11.78              0.395202   
4    2016      Chicago Bulls  27.562500          11.24              0.375663   

   injury_rate  
0     0.352941  
1     0.319360  
2     0.370875  
3     0.378049  
4     0.352134  
Shape: (300, 6)


Now that I have calculated the strength of schedule for each team in each season and the player stats needed, I will merge the strength of schedule dataframe to the current team_df. This will make it so that I have a front_office_df that includes payroll, draft order, and coaches along with a team_df that will contain team records, strength of schedule, and average stats.

In [919]:
# Merge on SOS and current team_df
team_df = pd.merge(
    team_df,
    sos_df,
    on=["Team", "Season"],
    how="inner"  
)

# Merge Player Stats and current team_df
team_df = pd.merge(
    team_df,
    player_stats_df,
    on=["Team", "Season"],
    how="inner"  
)

# Check result
print(team_df.head())
print(f"-----")
print(f"The shape of the dataframe:", team_df.shape)

   Season           Team  GP   W   L   WIN%   Min    PTS   FGM   FGA  ...  \
0    2016  Atlanta Hawks  82  48  34  0.585  48.4  102.8  38.6  84.4  ...   
1    2017  Atlanta Hawks  82  43  39  0.524  48.5  103.2  38.1  84.4  ...   
2    2018  Atlanta Hawks  82  24  58  0.293  48.1  103.4  38.2  85.5  ...   
3    2019  Atlanta Hawks  82  29  53  0.354  48.4  113.3  41.4  91.8  ...   
4    2020  Atlanta Hawks  67  20  47  0.299  48.6  111.8  40.6  90.6  ...   

   W_L  Pre-ASG_W  Pre-ASG_L  Post-ASG_W  Post-ASG_L       SOS    avg_age  \
0   11         31         24          17          10  0.500061  27.470588   
1   17         32         24          15          11  0.489720  28.200000   
2   12         18         41          17           6  0.508341  25.500000   
3   17         19         39          14          10  0.499671  25.136364   
4    9         15         41           6           5  0.509597  25.761905   

   avg_pts_top10  avg_production_score  injury_rate  
0          10.52    

The code below is ran to make sure that there are no data leaks within the team dataframe.

In [920]:
teams_2024 = team_df[team_df["Season"] == 2024]["Team"].unique()
teams_2025 = team_df[team_df["Season"] == 2025]["Team"].unique()

print("Missing in 2024:", set(nba_teams) - set(teams_2024))
print("Missing in 2025:", set(nba_teams) - set(teams_2025))

Missing in 2024: set()
Missing in 2025: set()


In order to get a clean target and feature variables, I am forced to merge the two master datasets I have into one master dataset.

In [921]:
# Merge team_df and front_office_df
master_df_unscaled = pd.merge(
    team_df,
    front_office_df,
    on=["Team", "Season"],
    how="inner"
)

print(master_df_unscaled.shape)
print(master_df_unscaled.head())
print(master_df_unscaled.info())

(328, 56)
   Season           Team  GP   W   L   WIN%   Min    PTS   FGM   FGA  ...  \
0    2016  Atlanta Hawks  82  48  34  0.585  48.4  102.8  38.6  84.4  ...   
1    2017  Atlanta Hawks  82  43  39  0.524  48.5  103.2  38.1  84.4  ...   
2    2018  Atlanta Hawks  82  24  58  0.293  48.1  103.4  38.2  85.5  ...   
3    2019  Atlanta Hawks  82  29  53  0.354  48.4  113.3  41.4  91.8  ...   
4    2020  Atlanta Hawks  67  20  47  0.299  48.6  111.8  40.6  90.6  ...   

              Coach  Yw/Franch  YOverall  CareerW  CareerL  CareerW%  \
0  Mike Budenholzer          3         3      146      100     0.593   
1  Mike Budenholzer          4         4      189      139     0.576   
2  Mike Budenholzer          5         5      213      197     0.520   
3      Lloyd Pierce          1         1       29       53     0.354   
4      Lloyd Pierce          2         2       49      100     0.329   

   FirstRoundPicks  SecondRoundPicks  Coach_Count      Payroll  
0                1           

Now that I have all of my dataframes merged into one master, I need to do some preprocessing of the data; more specifically, I need to scale the 2020 and 2021 shortened covid seasons to have 82 games played for only the totals,

In [922]:
# --- Step 1: Identify columns ---
id_cols = ["Season", "Team", "Coach", "Rk", "Pk", "Coach_Count", "GP"]
rate_cols = ["WIN%", "FG%", "3P%", "FT%", "SOS", "CareerW%", "injury_rate", "Payroll"]
career_cols = ["Yw/Franch", "YOverall", "CareerW", "CareerL"]  # career aggregates

# Columns eligible for scaling
counting_cols = [
    col for col in master_df_unscaled.columns
    if col not in id_cols + rate_cols + career_cols
]

# --- Step 2: Apply scaling ---
def scale_row(row):
    season_games = row["GP"]
    if season_games < 82:  # only scale shortened seasons
        scale_factor = 82 / season_games
        row[counting_cols] = row[counting_cols] * scale_factor
    return row

master_df = master_df_unscaled.apply(scale_row, axis=1)

# --- Step 3: Sanity check ---
print(master_df.loc[master_df["Season"].isin([2020, 2021]),
                          ["Season", "Team", "GP", "W", "L", "PTS", "avg_production_score"]].head())

    Season            Team  GP          W          L         PTS  \
4     2020   Atlanta Hawks  67  24.477612  57.522388  136.829851   
5     2021   Atlanta Hawks  72  46.694444  35.305556  129.491667   
6     2021   Atlanta Hawks  72  46.694444  35.305556  129.491667   
17    2020  Boston Celtics  72  54.666667  27.333333  129.491667   
18    2021  Boston Celtics  72  41.000000  41.000000  128.238889   

    avg_production_score  
4               0.466780  
5               0.471217  
6               0.471217  
17              0.481876  
18              0.451757  


In [923]:
# --- Create NWins column (next season's wins) ---
master_df = master_df.sort_values(by=["Team", "Season"]).reset_index(drop=True)

# Shift W column back one season within each team group
master_df["NWins"] = master_df.groupby("Team")["W"].shift(-1)

# For the latest season (e.g., 2025), NWins will naturally be NaN
print(master_df[["Season", "Team", "W", "NWins", "Coach", "FirstRoundPicks","SecondRoundPicks"]].tail(20))


     Season                Team          W      NWins          Coach  \
308    2017           Utah Jazz  51.000000  48.000000    Quin Snyder   
309    2018           Utah Jazz  48.000000  50.000000    Quin Snyder   
310    2019           Utah Jazz  50.000000  50.111111    Quin Snyder   
311    2020           Utah Jazz  50.111111  59.222222    Quin Snyder   
312    2021           Utah Jazz  59.222222  49.000000    Quin Snyder   
313    2022           Utah Jazz  49.000000  37.000000    Quin Snyder   
314    2023           Utah Jazz  37.000000  31.000000     Will Hardy   
315    2024           Utah Jazz  31.000000  17.000000     Will Hardy   
316    2025           Utah Jazz  17.000000        NaN     Will Hardy   
317    2016  Washington Wizards  41.000000  49.000000  Randy Wittman   
318    2017  Washington Wizards  49.000000  43.000000   Scott Brooks   
319    2018  Washington Wizards  43.000000  32.000000   Scott Brooks   
320    2019  Washington Wizards  32.000000  28.472222   Scott Br

The last step I have before I can start my final cleaning checklist is to make the master dataframes into csv files.

In [924]:
# # THE CODE TO CREATE RAW MASTER DATA CSV FILES
# # Define output folder
# output_dir = "/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/master-stats"
# os.makedirs(output_dir, exist_ok=True)

# # Save team_df
# team_path = os.path.join(output_dir, "team_df.csv")
# team_df.to_csv(team_path, index=False)
# print(f"team_df saved to {team_path}")

# # Save front_office_df
# front_office_path = os.path.join(output_dir, "front_office_df.csv")
# front_office_df.to_csv(front_office_path, index=False)
# print(f"front_office_df saved to {front_office_path}")

# # Save master_df
# master_path = os.path.join(output_dir, "master_df.csv")
# master_df.to_csv(master_path, index=False)
# print(f"master_df saved to {master_path}")

In [925]:
# # THE CODE TO CREATE PROCESSED MASTER DATA GZIP FILES
# # gzip file creation block
# root_dir = "/Users/trustanprice/Desktop/Personal/Basketball-Predictions/data/raw/master-stats"

# for subdir, _, files in os.walk(root_dir):
#     for file in files:
#         if file.endswith(".csv"):
#             file_path = os.path.join(subdir, file)
#             gz_path = file_path + ".gz"

#             try:
#                 df = pd.read_csv(file_path)

#                 if df.empty:
#                     print(f"Skipping empty file: {file_path}")
#                     continue

#                 print(f"Compressing {file_path} -> {gz_path}")
#                 df.to_csv(gz_path, index=False, compression="gzip")

#             except pd.errors.EmptyDataError:
#                 print(f"Skipping empty file (no data): {file_path}")


## Final Data Cleaning

Now that I have my main dataframes completed, I now need to go through the basic data cleaning checklist:
- checking for NaN values
- checking for duplicate values
- verify manually entered data (Season columns and Team names)

In [926]:
def cleaning_report(df, df_name="DataFrame"):
    print(f"--- Cleaning Report for {df_name} ---")
    
    # 1. Check for NaN values
    nan_summary = df.isna().sum()
    nan_summary = nan_summary[nan_summary > 0]
    if nan_summary.empty:
        print("No NaN values found")
    else:
        print("NaN values detected:")
        print(nan_summary)
    
    # 2. Check for duplicate rows
    dup_count = df.duplicated().sum()
    if dup_count == 0:
        print("No duplicate rows found")
    else:
        print(f"Found {dup_count} duplicate rows")

    # 3. Verify manually entered data (Season & Team)
    if "Season" in df.columns:
        seasons = df["Season"].unique()
        print(f"Unique seasons: {seasons}")
    else:
        print("No 'Season' column found")
        
    if "Team" in df.columns:
        teams = df["Team"].unique()
        print(f"Unique teams: {len(teams)} teams")
        print(sorted(teams))
    else:
        print("No 'Team' column found")
    
    print("-----------------------------------\n")

In [927]:
cleaning_report(front_office_df, "Front Office Data")
cleaning_report(sos_df, "Strength of Schedule")
cleaning_report(master_df, "Master Dataframe")

--- Cleaning Report for Front Office Data ---
No NaN values found
No duplicate rows found
Unique seasons: [2025 2024 2023 2022 2021 2020 2019 2018 2017 2016]
Unique teams: 30 teams
['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
-----------------------------------

--- Cleaning Report for Strength of Schedule ---
No NaN values found
No duplicate rows found
Unique seasons: [2025 2024 2023 2022 2021 2020 2019 2018 2017 2016]
Unique teams: 30 t

In [928]:
# Filter to seasons other than 2025
non_2025 = master_df[master_df["Season"] != 2025]

# Select only rows with at least one NaN
rows_with_nans = non_2025[non_2025.isna().any(axis=1)]

print(rows_with_nans.head())   # Preview first few rows
print("Total rows with NaN (non-2025):", len(rows_with_nans))


Empty DataFrame
Columns: [Season, Team, GP, W, L, WIN%, Min, PTS, FGM, FGA, FG%, 3PM, 3PA, 3P%, FTM, FTA, FT%, OREB, DREB, REB, AST, TOV, STL, BLK, BLKA, PF, PFD, PLUS_MINUS, Rk, Home_W, Home_L, Road_W, Road_L, E_W, E_L, W_W, W_L, Pre-ASG_W, Pre-ASG_L, Post-ASG_W, Post-ASG_L, SOS, avg_age, avg_pts_top10, avg_production_score, injury_rate, Coach, Yw/Franch, YOverall, CareerW, CareerL, CareerW%, FirstRoundPicks, SecondRoundPicks, Coach_Count, Payroll, NWins]
Index: []

[0 rows x 57 columns]
Total rows with NaN (non-2025): 0


This cleaning report has proved to be an immensely useful function. It was able to alert me that I had 61 NaN values in my Front Office dataset resulting in 61 NaN values in my Master dataset. In the end, it was a manual data insertion spelling error and has been resolved!

Now that I have finished cleaning dataframes, I will move onto the exploration notebook. In this notebook, I will be exploring the data through summary statistics and visualizations with the core goal of understanding the data as best as possible.